### Get started with PostGIS and Plotly
Assumptions: <br>
- PostgreSQL is installed locally
- You are in the directory that contains `montreal_places` folder as an example

Start your postgres server by running something like <br>
$ postgres -D /usr/local/pgsql/data

In [1]:
# Create a new postgreSQL database called 'postgis'
!createdb postgis

createdb: database creation failed: ERROR:  database "postgis" already exists


In [2]:
# add postgis language to the postgis database
!createlang plpgsql dc
# will get the following if it is there already :
# $ language "plpgsql" is already installed in database "postgis"

createlang: language "plpgsql" is already installed in database "postgis"


In [3]:
# install the postgis extensions to the postgis database
!psql -d postgis -c "CREATE EXTENSION postgis;"
!psql -d postgis -c "CREATE EXTENSION postgis_topology;"
# will get the following if they are already installed:
# ERROR:  extension "postgis" already exists
# ERROR:  extension "postgis_topology" already exists

ERROR:  extension "postgis" already exists
ERROR:  extension "postgis_topology" already exists


In [4]:
!cd montreal_places && ls

places.cpg places.dbf places.prj places.shp places.shx


In [ ]:
# Import shapefile
!shp2pgsql -c -D -s 4269 -I montreal_places/places.shp montreal_places | psql -d postgis

Start up the Plotly Database Application and connect to the `postgis` database. 

In [5]:
import requests

Make sure the connector is responsive by authenticating your connection.

In [6]:
auth = requests.get('http://localhost:5000/v1/authenticate')

In [7]:
auth.json()

{u'error': None}

Select your postgis database.

In [8]:
connectDatabase = requests.get('http://localhost:5000/v1/selectdatabase?database=postgis')

In [9]:
connectDatabase.json()

{u'error': None}

Make sure `montreal_places` table is there.

In [12]:
tables = requests.get('http://localhost:5000/v1/tables')
tables.json()

{u'error': None,
 u'tables': [{u'geography_columns': {}},
  {u'geometry_columns': {}},
  {u'spatial_ref_sys': {}},
  {u'raster_columns': {}},
  {u'raster_overviews': {}},
  {u'us_cities_2014': {}},
  {u'postgis': {}},
  {u'dc_census_tracts': {}},
  {u'dc_census_data': {}},
  {u'montreal_places': {}}]}

Looks like they are there!

Now, let's place a query to our postgis database to receive the locations of bouroughs around Montreal.

In [27]:
data = requests.get('http://localhost:5000/v1/query?statement=SELECT * FROM montreal_places LIMIT 100')

In [28]:
locations = data.json()['raw']
locations

[{u'geom': {u'coordinates': [-73.48916, 45.41746], u'type': u'Point'},
  u'gid': 1,
  u'name': u'La Prairie',
  u'osm_id': 93887876,
  u'population': None,
  u'type': u'town'},
 {u'geom': {u'coordinates': [-73.80674, 45.45667], u'type': u'Point'},
  u'gid': 2,
  u'name': u'Pointe-Claire',
  u'osm_id': 96699164,
  u'population': None,
  u'type': u'town'},
 {u'geom': {u'coordinates': [-73.75108, 45.4453], u'type': u'Point'},
  u'gid': 3,
  u'name': u'Dorval',
  u'osm_id': 98875745,
  u'population': None,
  u'type': u'town'},
 {u'geom': {u'coordinates': [-73.8679, 45.42891], u'type': u'Point'},
  u'gid': 4,
  u'name': u'Beaconsfield',
  u'osm_id': 98879474,
  u'population': None,
  u'type': u'town'},
 {u'geom': {u'coordinates': [-73.86475, 45.45291], u'type': u'Point'},
  u'gid': 5,
  u'name': u'Kirkland',
  u'osm_id': 98882241,
  u'population': None,
  u'type': u'town'},
 {u'geom': {u'coordinates': [-73.848, 45.49524], u'type': u'Point'},
  u'gid': 6,
  u'name': u'Pierrefonds-Roxboro',
 

In [30]:
geometries = [location['geom'] for location in locations]
geometries

[{u'coordinates': [-73.48916, 45.41746], u'type': u'Point'},
 {u'coordinates': [-73.80674, 45.45667], u'type': u'Point'},
 {u'coordinates': [-73.75108, 45.4453], u'type': u'Point'},
 {u'coordinates': [-73.8679, 45.42891], u'type': u'Point'},
 {u'coordinates': [-73.86475, 45.45291], u'type': u'Point'},
 {u'coordinates': [-73.848, 45.49524], u'type': u'Point'},
 {u'coordinates': [-73.91536, 45.41742], u'type': u'Point'},
 {u'coordinates': [-73.94614, 45.40928], u'type': u'Point'},
 {u'coordinates': [-73.9572, 45.43867], u'type': u'Point'},
 {u'coordinates': [-73.87014, 45.48047], u'type': u'Point'},
 {u'coordinates': [-73.51638, 45.38679], u'type': u'Point'},
 {u'coordinates': [-73.46789, 45.45548], u'type': u'Point'},
 {u'coordinates': [-73.99066, 45.38122], u'type': u'Point'},
 {u'coordinates': [-73.75878, 45.36371], u'type': u'Point'},
 {u'coordinates': [-73.68052, 45.30041], u'type': u'Point'},
 {u'coordinates': [-73.48213, 45.36178], u'type': u'Point'},
 {u'coordinates': [-73.54185,

In [31]:
geojsons = [{
    "type": "FeatureCollection",
    "features": [{
        "type": "Feature",
        "properties": {},
        "geometry": {
            "type": "GeometryCollection",
            "geometries": [geometry]
        }
    }]
} for geometry in geometries]

In [32]:
plotly_hq = dict(
            lon='-73.597379',
            lat='45.527937'
        )

In [33]:
import plotly.plotly as py
import plotly.tools as tls
from plotly.graph_objs import *

In [34]:
mapbox_access_token = 'pk.eyJ1IjoiY2hyaWRkeXAiLCJhIjoiRy1GV1FoNCJ9.yUPu7qwD_Eqf_gKNzDrrCQ'

In [35]:
data = Data([
    Scattermapbox(
        lat=['45.527937'],
        lon=['-73.597379'],
        mode='markers',
        marker=Marker(
            size=10
        ),
        text=['Plotly HQ'],
    )
])

In [36]:
layout = Layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        center=dict(
            lat=45.5,
            lon=-73.6
        ),
        pitch=0,
        zoom=8,
        layers=[
            {
                'sourcetype':'geojson',
                'source': geojson,
                'type':'symbol'
            } for geojson in geojsons
        ]
    )
)

In [37]:
fig = dict(data=data, layout=layout)

In [38]:
# tls.set_credentials_file(username='alexandres', api_key='')

In [39]:
# py.iplot(fig, filename='montreal_places') # this goes into 138

In [40]:
tls.embed("https://plot.ly/~alexandres/138/")